In [33]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [38]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import json
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [39]:
chrome_options = Options()
driver_path = '/Users/arundhutichakraborty/Downloads/chromedriver-mac-x64/chromedriver'
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

In [40]:
driver.get("https://www.99acres.com/real-estate-city-insights-lrffid?preference=RENTAL&src_page=HP")

In [41]:
city_elements = driver.find_elements(By.CLASS_NAME, 'ccd__heading')
city_names = [element.text.replace("Localities in ", "") for element in city_elements]
print(city_elements)
print(city_names)
city_urls = []
for element in city_elements:
    city_urls.append(element.get_attribute('href'))
print(city_urls)

[<selenium.webdriver.remote.webelement.WebElement (session="4b26407df976c52ea1af2aa651060734", element="f.03ACC637FB9050DEC89D0C8321C414B8.d.1937E2981675ED2E94B7D949626C0D71.e.8")>, <selenium.webdriver.remote.webelement.WebElement (session="4b26407df976c52ea1af2aa651060734", element="f.03ACC637FB9050DEC89D0C8321C414B8.d.1937E2981675ED2E94B7D949626C0D71.e.9")>, <selenium.webdriver.remote.webelement.WebElement (session="4b26407df976c52ea1af2aa651060734", element="f.03ACC637FB9050DEC89D0C8321C414B8.d.1937E2981675ED2E94B7D949626C0D71.e.10")>, <selenium.webdriver.remote.webelement.WebElement (session="4b26407df976c52ea1af2aa651060734", element="f.03ACC637FB9050DEC89D0C8321C414B8.d.1937E2981675ED2E94B7D949626C0D71.e.11")>, <selenium.webdriver.remote.webelement.WebElement (session="4b26407df976c52ea1af2aa651060734", element="f.03ACC637FB9050DEC89D0C8321C414B8.d.1937E2981675ED2E94B7D949626C0D71.e.12")>, <selenium.webdriver.remote.webelement.WebElement (session="4b26407df976c52ea1af2aa651060734

In [42]:
with open('city_urls.json', 'w') as file:
    json.dump(city_urls, file)
    
print(f"Extracted {len(city_urls)} city URLs.")
driver.quit()

Extracted 15 city URLs.


In [44]:
def initialize_driver():
    chrome_options = Options()
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    return webdriver.Chrome(options=chrome_options)

In [47]:
with open('city_urls.json', 'r') as file:
    city_urls = json.load(file)
    
city_localities = {}
city_localities_elements = {}

In [48]:
for url in city_urls:
    print(f"Processing URL: {url}")
    city_localities_properties_urls = []
    driver = initialize_driver()
    
    try:
        driver.get(url)

        # Wait for locality elements to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'section_header_semiBold'))
        )

        # Find and store localities
        locality_elements = driver.find_elements(By.CLASS_NAME, 'section_header_semiBold')
        
        localities = [element.text for element in locality_elements]
        for element in city_elements:
            city_localities_properties_urls.append(element.get_attribute('href'))
            
        print(city_localities_properties_urls)
        
        city_name = url.split('/')[-1].split('-lrffid')[0]  # Extract city name from URL (adjust as necessary)
        city_localities[city_name] = localities
        city_localities_elements[city_name] = locality_elements
        

    except Exception as e:
        print(f"Error processing URL {url}: {e}")

    finally:
        # Close the driver
        driver.quit()
    
    # Pause to avoid overloading the server
    time.sleep(5)

Processing URL: https://www.99acres.com/localities-in-bangalore-lrffid
Error processing URL https://www.99acres.com/localities-in-bangalore-lrffid: HTTPConnectionPool(host='localhost', port=56283): Max retries exceeded with url: /session/4b26407df976c52ea1af2aa651060734/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x160555910>: Failed to establish a new connection: [Errno 61] Connection refused'))
Processing URL: https://www.99acres.com/localities-in-delhi-lrffid
Error processing URL https://www.99acres.com/localities-in-delhi-lrffid: HTTPConnectionPool(host='localhost', port=56283): Max retries exceeded with url: /session/4b26407df976c52ea1af2aa651060734/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x160580210>: Failed to establish a new connection: [Errno 61] Connection refused'))
Processing URL: https://www.99acres.com/localities-in-hyderabad-lrffid
Error processing URL https://www.99acres.com/lo

Processing URL: https://www.99acres.com/localities-in-ghaziabad-lrffid
Error processing URL https://www.99acres.com/localities-in-ghaziabad-lrffid: Message: 
Stacktrace:
0   chromedriver                        0x00000001052e1208 cxxbridge1$str$ptr + 1927396
1   chromedriver                        0x00000001052d966c cxxbridge1$str$ptr + 1895752
2   chromedriver                        0x0000000104ed4808 cxxbridge1$string$len + 89564
3   chromedriver                        0x0000000104f18bcc cxxbridge1$string$len + 369056
4   chromedriver                        0x0000000104f52228 cxxbridge1$string$len + 604156
5   chromedriver                        0x0000000104f0d698 cxxbridge1$string$len + 322668
6   chromedriver                        0x0000000104f0e310 cxxbridge1$string$len + 325860
7   chromedriver                        0x00000001052a7e78 cxxbridge1$str$ptr + 1693012
8   chromedriver                        0x00000001052ac77c cxxbridge1$str$ptr + 1711704
9   chromedriver             

In [49]:
# Save the localities data to a file
with open('city_localities.json', 'w') as file:
    json.dump(city_localities, file)
    
with open('city_localities_elements.json', 'w') as file:
    json.dump(city_localities_elements, file)


print("Data saved to city_localities.json.")

Data saved to city_localities.json.


In [3]:
import json
import csv

# Function to load JSON data and save it to final.csv
def save_json_to_csv(json_file, csv_file='final.csv'):
    # Load the JSON data from the file
    with open(json_file, 'r', encoding='utf-8') as file:
        city_localities = json.load(file)

    # Open the CSV file for writing
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Write the header
        writer.writerow(['City', 'Locality', 'Price per sq.ft'])
        
        # Write the data
        for city, localities in city_localities.items():
            for i in range(0, len(localities), 2):
                locality = localities[i]
                price = localities[i+1] if i+1 < len(localities) else 'NA'
                writer.writerow([city.split('-in-')[-1].capitalize(), locality, price])
    
    print(f"Data saved to {csv_file}")

# Call the function with your JSON file
save_json_to_csv('city_localities.json', 'Day 2 Final.csv')


Data saved to Day 2 Final.csv
